<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=6b221e3a73c558dccc7e9570e64158aa2214eddaaec85335703fd290ca2021a8
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib
Cloning into 'Project9_data'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 67 (delta 20), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (67/67), 1.65 MiB 

In [ ]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re

!ls Project9_data


 expressen.txt	'Konsten att fejka arabiska.htm'   Project.ipynb


In [ ]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu".lower()
  CoNLL.write_doc2conll(doc, conllufilename)

  return conllufilename

In [ ]:
class SemanticAnalyzer:
  def __init__(self, file_path):
    self.file_path = file_path
    self.data = self.parse_conll_u(self.file_path)
    self.PFL_sentence_matches = self.person_first_lang(self.data)
    self.IFL_sentence_matches = self.identity_first_lang(self.data)

  def parse_conll_u(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
      data = {}
      sentence = []
      for line in file:
        line = line.strip()
        if line.startswith('# text'):
          sentence = []
        if line.startswith("# sent_id"):
          line = line.split("=")
          key = line[1].strip()
          sentence = []
        elif line == '':
          if sentence:
            data[key] = sentence
        else:
            sentence.append(line.split('\t'))
    return data

  def person_first_lang(self, data):
    PFL_sents = {}
    PFL_regx = re.compile(r"har|med", re.IGNORECASE)

    for id, sentence in data.items():
      for token in sentence:
        if token[1] == "autism" or token[1].lower() == "autismspektrumtillstånd" or token[1].lower() == "ast":
          if sentence[int(token[0])-2][1] == "och".lower():
            if PFL_regx.match(sentence[int(sentence[int(token[6])-1][int(6)])-1][1]):
              PFL_sents[id] = sentence
          if PFL_regx.match(sentence[int(token[0])-2][1]):
            PFL_sents[id] = sentence
    return PFL_sents

  def identity_first_lang(self, data):
    IFL_sents = {}
    IFL_regx = re.compile(r"^.*(autistisk|autistiska)", re.IGNORECASE)

    for id, sentence in data.items():
      for token in sentence:
        if token[1] == "spektrumet" or token[1] == "autismspektrumet":
          if sentence[int(token[6])][1].lower() == "på":
            IFL_sents[id] = sentence
            continue
        if IFL_regx.match(token[1]):
          IFL_sents[id] = sentence

    return IFL_sents

####

def main(file, lang):
  analyzer = SemanticAnalyzer(file_path= file)
  if lang.lower() == "pfl":
    matches = analyzer.PFL_sentence_matches
  elif lang.lower() == "ifl":
    matches = analyzer.IFL_sentence_matches
  else:
    print("Invalid language choice. Please choose either 'PFL' or 'IFL'.")
    return

  for id, sentence in matches.items():
    print(f"Sentence ID: {int(id) + 1} | {' '.join(token[1] for token in sentence)}")
    print("")

  print(f"Total matching sentences: {len(matches)}")

if __name__ == "__main__":
    file_path = input("Enter the file path: ").lower()
    if ".conllu" not in file_path:
      #tag_and_convert(file_path)
      file_path = tag_and_convert(file_path)
    lang = input("Enter the language (PFL or IFL): ")
    main(file_path, lang)

Enter the file path: Konsten att fejka arabiska.htm
/content/Project9_data
/content/Project9_data
Enter the language (PFL or IFL): ifl
Total matching sentences: 0
